# Exploring spatial filtering, low pass
Joshua Stough
DIP 3.4 and 3.5 on spatial filtering and low-pass. See [correlate](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.correlate.html)

Here we're going to look at a couple of spatial filters that reduce high frequency information. In other words, they blur the image.
- First is a simple averaging, as we've done before. 
- Second is a Gaussian, which gives more weight to pixels closer to the center and less farther away. The Gaussian has better properties with respect to frequency information. We've seen Gaussian histograms, but you can read more about them in DIP 2.6.

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

# For spatial filtering/operations
from scipy.ndimage import (correlate,
                           convolve)

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import (arr_info,
                          make_linmap)
from vis_utils import (vis_rgb_cube,
                       vis_hists,
                       vis_pair)

from skimage.filters import *

plt.style.use('dark_background')

In [ ]:
I = plt.imread('../dip_pics/cat_small.png').astype('float')
arr_info(I)

In [ ]:
vis_hists(I)

In [ ]:
# The image includes an alpha channel that we're not going to need.
I = I[...,:3].copy()
arr_info(I)

&nbsp;

## Low-pass filtering, or blurring, involves averaging neighborhoods of pixels,
thereby reducing the difference between neighboring pixels. We can do this in a number of ways. We've already seen local, equal weight averaging. We're going to use a relatively large kernel size (filter size) to see the differences between the filters.

Note also, we'll apply the filter to each channel separately. Zoom in on the figure to see the local differences.

In [ ]:
mean_h = np.ones((11,11))/121
# mean_h

In [ ]:
I_localAverage11 = np.stack([correlate(I[...,i], mean_h) for i in range(3)], axis=-1)

In [ ]:
vis_pair(I, I_localAverage11)

&nbsp;

## We can also try a weighted average that more highly weights 
the pixels in the center of the neighborhood. A Gaussian is a good choice here.

In [ ]:
from scipy.signal import gaussian
gaus_h = gaussian(11, std=2, sym=True) # A Gaussian with std 1 symmetric about the size M.
# The result is a 1-d thing. take it's outer product to make a 2d thing.
gaus_h = np.outer(gaus_h, gaus_h)
gaus_h = gaus_h/gaus_h.sum()

In [ ]:
# plot the two filter types
plt.figure()
plt.plot(gaus_h[5,:], label='Gaussian')
plt.plot(mean_h[5,:], label='Average')
plt.legend()
plt.suptitle('Different local filters');

In [ ]:
# Or consider in 2D.
vis_pair(gaus_h, mean_h, first_title='Gaussian', second_title='Average')

In [ ]:
I_localGaussian11 = np.stack([correlate(I[...,i], gaus_h) for i in range(3)], axis=-1)

In [ ]:
# Vis all three.
f, ax = plt.subplots(1,3, figsize=(8,3), sharex=True, sharey=True)
ax[0].imshow(I)
ax[0].set_title('Original')

ax[1].imshow(I_localAverage11)
ax[1].set_title('Average')

ax[2].imshow(I_localGaussian11)
ax[2].set_title('Gaussian')

plt.suptitle('Different Blurs of Cat')

&nbsp;

## Let's consider a midline through the image, and see what the
two filters did to the intensity values.

In [ ]:
# Let's look at a midline through the image.
s = I.shape

f, ax = plt.subplots(1,2, figsize=(8,4))
ax[0].imshow(I)
ax[0].hlines(s[0]//2, xmin=0, xmax=s[1], color='r')
ax[0].set_title('Sample red channel from line.')

ax[1].plot(I[s[0]//2,:,0], label='Original')
ax[1].plot(I_localAverage11[s[0]//2,:,0], label='Average')
ax[1].plot(I_localGaussian11[s[0]//2,:,0], label='Gaussian')
ax[1].legend(loc='lower right')

&nbsp;

## Interactive Visualization 

We're going to interactively visualize the Gaussian blur as you change the sigma/width of the gaussian.

See [jupyter-matplotlib](https://github.com/matplotlib/jupyter-matplotlib/blob/master/examples/ipympl.ipynb)

In [ ]:
from scipy.signal import gaussian
def make_gaus(size=25, sig=1):
    gaus_h = gaussian(size, std=sig, sym=True) # A Gaussian with std 1 symmetric about the size M.
    # The result is a 1-d thing. take it's outer product to make a 2d thing.
    gaus_h = np.outer(gaus_h, gaus_h)
    gaus_h = gaus_h/gaus_h.sum()
    return gaus_h

In [ ]:
from ipywidgets import VBox, FloatSlider, FloatLogSlider

sig = 0.5
gaus_h = make_gaus(sig=sig)
I_g = np.stack([correlate(I[...,i], gaus_h) for i in range(3)], axis=-1)


plt.ioff()
plt.clf()

# https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#FloatLogSlider
slider = FloatSlider(
    orientation='horizontal',
    value=0.5,
    min=0.5,
    max=20.0,
    step=0.5,
    description='Sig'
)

fig_args = {'num':' ', 'frameon':True}
fig, ax = plt.subplots(1,3, figsize=(10,3), **fig_args) 

# display artists I'll update
adisp = ax[0].imshow(I)
hdisp = ax[1].imshow(gaus_h, cmap='magma', vmin=0, vmax=gaus_h.max())
gdisp = ax[2].imshow(I_g)

htext = ax[1].set_title('Sig {}'.format(sig))
gtext = ax[2].set_title('Sig {}'.format(sig))


def update_image(change):
    global I, I_g, gaus_h, adisp, hdisp, gdisp, htext, gtext
    sig = change.new
    gaus_h = make_gaus(sig=sig)
    I_g = np.stack([correlate(I[...,i], gaus_h) for i in range(3)], axis=-1)
    
    
    hdisp.set_array(gaus_h)
    # need to reset the color limits each time since gaus_h is changing a lot. 
    hdisp.set_clim(0.0,gaus_h.max()) 
    htext.set_text('Sig {}'.format(sig))
    
    gdisp.set_array(I_g)
    gtext.set_text('Sig {}'.format(sig))
    
    fig.canvas.draw()
    fig.canvas.flush_events()

slider.observe(update_image, names='value')

VBox([slider, fig.canvas])